In [71]:
import os
import time
import pandas as pd
# import modin.pandas as pd
from dask import dataframe as dask
import ray as r
import logging
import os
import subprocess
import yaml

import datetime 
import gc
import re
import re
import datetime
import csv
import gzip
import yaml



In [2]:
# data = 
# gender_submission.csv
# os.path.getsize('gender_submission.csv')

3258

## Reading the data

#### With Modlin and Ray

In [72]:

r.shutdown()
r.init()
start = time.time()
df = pd.read_csv('gender_submission.csv')
begin = time.time()
print("Time taken to read: ",(begin-start),"sec")

2023-04-28 11:17:29,407	INFO worker.py:1625 -- Started a local Ray instance.


Time taken to read:  0.009287834167480469 sec


#### With Dask

In [73]:

begin = time.time()
dask_data = dask.read_csv('gender_submission.csv')
end = time.time()
print("Time taken to read: ",(begin-start),"sec")

Time taken to read:  134.20073699951172 sec


#### With Pandas

In [74]:

begin = time.time()
pandas_data = pd.read_csv('gender_submission.csv')
end = time.time()
print("Time taken to read: ",(begin-start),"sec")

Time taken to read:  136.75519514083862 sec


#### We see that dask reads the fastest with 0.0100 seconds followed by Modlin and Ray with and the least being Pandas with 99.3447 seconds

## Validation on Column

In [75]:
pandas_data.columns= pandas_data.columns.str.replace('[#,@,&]','')

/var/folders/lt/lr7tn3dj3jb6ltvh3_h48b8c0000gn/T/ipykernel_8702/378534539.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  pandas_data.columns= pandas_data.columns.str.replace('[#,@,&]','')


In [76]:
pandas_data.columns = df.columns.str.replace(' ', '')

In [78]:
pandas_data

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


### Creating  YAML File

In [25]:
%%writefile file.py

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.load(stream, Loader=yaml.Loader)
        except yaml.YAMLError as exc:
            logging.error(exc)

Writing file.py


### Method for Validating number of columns and column name of ingested file with YAML.

In [50]:

def col_header_val(df,table_config):
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: re.sub(r'\W+', '_', x), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("validation passed")
        return 1
    else:
        print("validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

### Creating a YAML file and writing the column name in YAML file

In [27]:
%%writefile file_2.yaml
file_type: csv
dataset_name: file
file_name: Rate
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Survived
    - PassengerId

Writing file_2.yaml


In [38]:

with open('file_2.yaml', 'r') as file:

    yaml_content = yaml.load(file, Loader=yaml.FullLoader)
yaml_content    

{'file_type': 'csv',
 'dataset_name': 'file',
 'file_name': 'Rate',
 'table_name': 'edsurv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'columns': ['Survived', 'PassengerId']}

In [55]:

df = pd.read_csv('gender_submission.csv',yaml_content['inbound_delimiter'])
df.columns

/Users/ujunwafatima/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3369: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  exec(code_obj, self.user_global_ns, self.user_ns)


Index(['PassengerId', 'Survived'], dtype='object')

### Validating number of columns and column name of ingested file with YAML.

In [79]:

col_header_val(df,yaml_content)

validation passed


1

In [80]:
print("columns of files are:" ,df.columns)
print("columns of YAML are:" ,yaml_content['columns'])

columns of files are: Index(['passengerid', 'survived'], dtype='object')
columns of YAML are: ['Survived', 'PassengerId']


### Writing csv in gz format in pipe separated text file (|)

In [81]:

df = dask.read_csv('gender_submission.csv',delimiter=',')


df.to_csv('gender_submission.csv.gz',
          sep='|',
          header=True,
          index=False,
          quoting=csv.QUOTE_ALL,
          compression='gzip',
          quotechar='"',
          doublequote=True,
          line_terminator='\n')

['/Users/ujunwafatima/Desktop/Data Glacier Assignments/Week6/gender_submission.csv.gz/0.part']

### Summary of the file

In [66]:
# no of files
no_of_files = os.listdir('/Users/ujunwafatima/Desktop/Data Glacier Assignments/Week6/gender_submission.csv.gz/')
for file in no_of_files:
    print(file)

0.part


In [67]:
#size of the gz format folder
os.path.getsize('/Users/ujunwafatima/Desktop/Data Glacier Assignments/Week6/gender_submission.csv.gz/')

96